In [1]:
'''This notebook implements the object tracking algorithm based on Harris corner detection
and basic Lucas Kanade optic flow.'''

import numpy as np
import cv2
import argparse

In [2]:
def cal_derivative(img1, img2):
    
    dx1 = cv2.Sobel(img1,cv2.CV_64F,1,0,ksize=3)
    dy1 = cv2.Sobel(img1,cv2.CV_64F,0,1,ksize=3)
    dx2 = cv2.Sobel(img2,cv2.CV_64F,1,0,ksize=3)
    dy2 = cv2.Sobel(img2,cv2.CV_64F,0,1,ksize=3)
    ix = (dx1 + dx2) / 2
    iy = (dy1 + dy2) / 2
    
    sigma = 1.0
    k = cv2.getGaussianKernel(9, sigma)
    g = k * np.transpose(k)
    imgFiltered1 = cv2.filter2D(img1, -1, g)
    imgFiltered2 = cv2.filter2D(img2, -1, g)
    it = imgFiltered1 - imgFiltered2
    
    return ix, iy, it

In [3]:
def harris_corner(img):
    ix = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=3)
    iy = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=3)
    
    k = 0.05
    sigma = 1.0
    g = cv2.getGaussianKernel(3, sigma)
    G = g * np.transpose(g)
    sum_filter = np.ones((3, 3))
    ixx = cv2.filter2D(ix**2, -1, G)
    iyy = cv2.filter2D(iy**2, -1, G)
    ixy = cv2.filter2D(ix*iy, -1, G)
    sxx = cv2.filter2D(ixx, -1, sum_filter)
    syy = cv2.filter2D(iyy, -1, sum_filter)
    sxy = cv2.filter2D(ixy, -1, sum_filter)
    det = sxx*syy-sxy**2
    trace = sxx + syy
    response = det - k*(trace**2)
    max_response = np.max(response)
    features = np.where(response>=1e-1*max_response)
    filtered_x = []
    filtered_y = []
    for i in range(len(features[0])):
        if i==0:
            filtered_x.append(features[0][i])
            filtered_y.append(features[1][i])
        else:
            bound = 0 if len(filtered_x)<=20 else (len(filtered_x)-20)
            flag = True
            for j in range(bound, len(filtered_x)):
                d = (features[0][i]-filtered_x[j])**2 + (features[1][i]-filtered_y[j])**2
                if d<=100:
                    flag = False
            if flag:
                filtered_x.append(features[0][i])
                filtered_y.append(features[1][i])
    return (np.array(filtered_x), np.array(filtered_y))

In [4]:
def LK_optic_flow(img1, img2, features=None):
    U = []
    V = []
    U_map = np.zeros(img1.shape)
    V_map = np.zeros(img1.shape)
    window_size = (5, 5)
    sum_kernel = np.ones(window_size)
    ix, iy, it = cal_derivative(img1, img2)
#     six = cv2.filter2D(ix, -1, sum_kernel)
#     siy = cv2.filter2D(iy, -1, sum_kernel)
#     sit = cv2.filter2D(it, -1, sum_kernel)
    
    if features is None:
        features = harris_corner(img1)
        
    for i in range(len(features[0])):
        center_x, center_y = features[0][i], features[1][i]
        offset_x = int(((window_size[1]-1)/2))
        offset_y = int(((window_size[0]-1)/2))
        if center_x < offset_x:
            center_x = offset_x
        if center_x > img1.shape[1] - (offset_x+1):
            center_x = img1.shape[1] - offset_x
        if center_y < offset_y:
            center_y = offset_y
        if center_y >img1.shape[0] - (offset_y+1):
            center_y = img1.shape[0] - offset_y
#         print(center_x-offset_x, center_x+offset_x, center_y-offset_y, center_y+offset_y)
        pix = ix[int(center_x-offset_x):int(center_x+offset_x), int(center_y-offset_y):int(center_y+offset_y)]
        piy = iy[int(center_x-offset_x):int(center_x+offset_x), int(center_y-offset_y):int(center_y+offset_y)]
        pit = it[int(center_x-offset_x):int(center_x+offset_x), int(center_y-offset_y):int(center_y+offset_y)]
        X = np.reshape(pix, (1, -1)).squeeze()
        Y = np.reshape(piy, (1, -1)).squeeze()
        b = np.reshape(pit, (1, -1)).squeeze()
        AT = np.array([X, Y])
        A = np.transpose(AT)
        ATA = np.matmul(AT, A)
        ATb = np.matmul(AT, b)
        ATA_inv = np.linalg.pinv(ATA)
        x = np.matmul(ATA_inv, ATb)
        U.append(x[0])
        V.append(x[1])
        
    for i in range(len(features[0])):
        U_map[features[0][i], features[1][i]] = U[i]
        V_map[features[0][i], features[1][i]] = V[i]
        
    return features, U_map, V_map

In [5]:
def estimate(features, u, v):
    # normalization
#     u[np.where(u==0)] = 1e-5
#     v[np.where(v==0)] = 1e-5
    threshold = 1e-3
    for i in range(u.shape[0]):
        for j in range(u.shape[1]):
            if u[i, j] > threshold:
                u[i, j] = 1
            elif u[i, j] < -threshold:
                u[i, j] = -1
            else:
                u[i, j] = 0
            if v[i, j] > threshold:
                v[i, j] = 1
            elif v[i, j] < -threshold:
                v[i, j] = -1
            else:
                v[i, j] = 0
    
    new_xs = []
    new_ys = []
    for i in range(len(features[0])):
#         print(i, features[0][i], features[1][i], u[features[0][i]][features[1][i]], v[features[0][i]][features[1][i]])
        new_x = features[0][i] + v[features[0][i]][features[1][i]]
        new_y = features[1][i] + u[features[0][i]][features[1][i]]
        if new_x >= u.shape[0]:
            new_x = u.shape[0]-1
        if new_y >= u.shape[1]:
            new_y = u.shape[1]-1
        new_xs.append(int(new_x))
        new_ys.append(int(new_y))
        
    return (new_xs, new_ys)

In [6]:
def video_record()->str:
    #call the camera of the computer
    cap = cv2.VideoCapture(0)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    #set the output Parameters
    out = cv2.VideoWriter('recording.avi', fourcc, 30.0, (640,480))
    while(cap.isOpened()):
        ret, frame=cap.read()
        if ret==True:
            frame = cv2.flip(frame, 1)
            out.write(frame)
            cv2.imshow('frame', frame)
            k = cv2.waitKey(30) & 0xff
            if k == 27:
                break
        else:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    return 'recording.avi'

cap = cv2.VideoCapture("demo7.mp4")

# random colors for plotting
color = np.random.randint(0,255,(100,3))

# feature extraction
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = harris_corner(old_gray)

# Create a mask image for plotting
mask = np.zeros_like(old_frame)

while(1):
    ret,frame = cap.read()
    if ret == False:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # calculate optical flow
    p1, u, v = LK_optic_flow(old_gray, frame_gray, p0)
    p1 = estimate(p1, u, v)

    # plottings
    for i in range(len(p1[0])):
        a,b = p1[1][i], p1[0][i]
        c,d = p0[1][i], p0[0][i]
        mask = cv2.line(mask, (int(a),int(b)),(int(c),int(d)), color[i].tolist(), 2)
        frame = cv2.circle(frame,(int(a),int(b)),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)
    cv2.imshow('frame',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # update features and frames
    old_gray = frame_gray.copy()
    p0 = p1
    
cv2.destroyAllWindows()